## Imports

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt

Cloning into 'Capstone'...
remote: Enumerating objects: 2938, done.
remote: Counting objects: 100% (612/612), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 2938 (delta 420), reused 559 (delta 367), pack-reused 2326
Receiving objects: 100% (2938/2938), 272.86 MiB | 18.21 MiB/s, done.
Resolving deltas: 100% (1872/1872), done.
Updating files: 100% (160/160), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 972.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.6 MB/s eta 

In [ ]:
import re
import datetime
import time
import pandas as pd
import os
import tqdm
from argparse import ArgumentParser
from argparse import Namespace

from datasets import load_dataset

import torch
from torch.nn.functional import softmax
import numpy as np

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("StabilityAI/stablelm-tuned-alpha-7b")
model = AutoModelForCausalLM.from_pretrained(
    "StabilityAI/stablelm-tuned-alpha-7b", device_map="auto", torch_dtype=torch.float16
)

# tokenizer = AutoTokenizer.from_pretrained("h2oai/h2ogpt-oasst1-512-20b", padding_side="left")
# model = AutoModelForCausalLM.from_pretrained("h2oai/h2ogpt-oasst1-512-20b", load_in_8bit=True, torch_dtype=torch.bfloat16, device_map="auto")

model.cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 6144)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=6144, out_features=18432, bias=True)
          (dense): Linear(in_features=6144, out_features=6144, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=6144, out_features=24576, bias=True)
          (dense_4h_to_h): Linear(in_features=24576, out_features=6144, bias=True)
          (act): GELUActivation()
        )
      )
      (1): GPTNeoXLayer(
        (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=Tr

In [ ]:
os.chdir("/content/Capstone/src/fact_completion_scripts")
# from probe_helpers import probe_gpt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if not torch.cuda.is_available():
    raise Exception("Change runtime type to include a GPU.")

In [ ]:
system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

In [ ]:
context = "Please complete the following sentence with an exact, succinct answer: What sport does Megan Rapinoe play? They play"

In [ ]:
context_with_preamble = f"{system_prompt}<|USER|>{context}<|ASSISTANT|>"

In [ ]:
target_id = tokenizer.encode("soccer", return_tensors="pt").to(device)[0][0]

In [ ]:
target_id

tensor(33353, device='cuda:0')

In [ ]:
# sanity check
tokenizer.batch_decode([target_id])

['soc']

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnTokens(StoppingCriteria):
    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [ ]:
def probe_stability(
    model, tokenizer, target_id, context, max_tokens_to_generate=1, verbose=False
):
    input_ids = tokenizer(context, return_tensors="pt").to(device)

    target_scalar = target_id.detach().cpu().numpy()

    outputs = model.generate(
        **input_ids,
        max_new_tokens=max_tokens_to_generate,
        do_sample=True,
        stopping_criteria=StoppingCriteriaList([StopOnTokens()]),
        return_dict_in_generate=True,
        output_scores=True,
    )

    output_sequences = outputs.sequences
    output_scores = outputs.scores
    logits = output_scores[0][-1]

    # convert our prediction scores to a probability distribution with softmax
    probs = softmax(logits, dim=-1)

    probs = list(probs.detach().cpu().numpy())

    if verbose:
        print(f"\n\tcontext... {context}")
        print(f"\ttokenized_context ids... {input_ids}")
        print(
            f"\tdecoded tokenized_context... {tokenizer.batch_decode(input_ids[0].ids)}"
        )
        print(f"\tdecoded target id... {tokenizer.decode([target_id.item()])}")
        print(
            f"\tmost probable prediction id decoded... {tokenizer.decode([np.argmax(probs)])}\n"
        )
        print(f"\tfull decoded output.... {tokenizer.batch_decode(output_sequences)}")

    # double check weird-ness before accessing prob
    if len(probs) < target_id:
        return None

    # return the likelihood that our stipulated target would follow the context,
    # according to the model
    try:
        return np.take(probs, [target_scalar])[0]

    except IndexError:
        print("target index not in model vocabulary scope; raising IndexError")
        return None

In [ ]:
# using the preamble'd prompt
probe_stability(
    model,
    tokenizer,
    target_id,
    context_with_preamble,
    max_tokens_to_generate=50,
    verbose=True,
)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



	context... <|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
<|USER|>Please complete the following sentence with an exact, succinct answer: What sport does Megan Rapinoe play? They play<|ASSISTANT|>
	tokenized_context ids... {'input_ids': tensor([[50277,     4,   659,   494, 22047, 27664,   264,   313, 29858,  2715,
            10,   187,    14,   659,   494, 22047,   310,   247,  9371,   285,
         25365,  1527,    14,  6756, 14980,  3448,  1566,  3715,   407,   659,
          1430, 18128,    15,   187,    14,   659,   494, 22047,   310,  9049,
           281,   320,  2104,   

0.0

In [ ]:
# using the normal prompt
probe_stability(model, tokenizer, target_id, context, verbose=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



	context... Please complete the following sentence with an exact, succinct answer: What sport does Megan Rapinoe play? They play
	tokenized_context ids... {'input_ids': tensor([[ 7845,  3426,   253,  1563,  6197,   342,   271,  3242,    13, 18382,
          4291,  3662,    27,  1737,  9678,  1057, 45867, 17314,  2610,    70,
          1132,    32,  1583,  1132]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
	decoded tokenized_context... ['Please', ' complete', ' the', ' following', ' sentence', ' with', ' an', ' exact', ',', ' succ', 'inct', ' answer', ':', ' What', ' sport', ' does', ' Megan', ' Rap', 'ino', 'e', ' play', '?', ' They', ' play']
	decoded target id... soc
	most probable prediction id decoded...  soccer

	full decoded output.... ['Please complete the following sentence with an exact, succinct answer: What sport does Megan Rapinoe play? They play basketball']


0.0